In [1]:
!git clone https://github.com/imatge-upc/speech2signs-2017-nmt.git

Cloning into 'speech2signs-2017-nmt'...
remote: Enumerating objects: 57, done.
remote: Total 57 (delta 0), reused 0 (delta 0), pack-reused 57
Unpacking objects: 100% (57/57), done.


In [2]:
!wget http://www.bu.edu/asllrp/dai-asllvd-BU_glossing_with_variations_HS_information-extended-urls-RU.xlsx

--2019-09-15 05:12:45--  http://www.bu.edu/asllrp/dai-asllvd-BU_glossing_with_variations_HS_information-extended-urls-RU.xlsx
Resolving www.bu.edu (www.bu.edu)... 13.249.134.39, 13.249.134.77, 13.249.134.47, ...
Connecting to www.bu.edu (www.bu.edu)|13.249.134.39|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1524218 (1.5M) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘dai-asllvd-BU_glossing_with_variations_HS_information-extended-urls-RU.xlsx’

dai-asllvd-BU_gloss 100%[===================>]   1.45M  1.24MB/s    in 1.2s    

2019-09-15 05:12:46 (1.24 MB/s) - ‘dai-asllvd-BU_glossing_with_variations_HS_information-extended-urls-RU.xlsx’ saved [1524218/1524218]



In [3]:
!ls

dai-asllvd-BU_glossing_with_variations_HS_information-extended-urls-RU.xlsx
sample_data
speech2signs-2017-nmt


In [0]:
import pandas as pd
import numpy as np

In [21]:
glosses_data.head()

,Unnamed: 0,Main New Gloss,Consultant,Main New Gloss.1,Gloss Variant,D Start HS,N-D Start HS,D End HS,N-D End HS,Passive Arm,Combined,Separate,Session,Scene,Start,End,Combined.1,Separate.1
0,NaN,============,============,============,============,============,============,============,============,============,============,============,============,============,============,============,============,============
1,NaN,TWENTY,============,============,============,============,============,============,============,============,============,============,============,============,============,============,============,============
2,NaN,NaN,Liz,TWENTY,TWENTY+,crvd-L,NaN,flat-G,NaN,N,NaN,MOV,ASL_2008_01_11,2,2635,2661,NaN,MOV
3,NaN,NaN,Tyler,TWENTY,TWENTY++,L,NaN,flat-G,NaN,N,MOV,MOV,ASL_2008_05_12a,1,2400,2480,MOV,MOV
4,NaN,NaN,Naomi,TWENTY,TWENTY+,L,NaN,baby-O,NaN,N,NaN,MOV,ASL_2008_08_04,1,2279,2353,NaN,MOV


In [0]:
glosses_data = pd.read_excel("dai-asllvd-BU_glossing_with_variations_HS_information-extended-urls-RU.xlsx")

In [0]:
removed = ['============', False]
glosses = glosses_data["Gloss Variant"].dropna().unique()
glosses = [gloss for gloss in glosses if gloss not in removed]
glosses = [gloss.rstrip("+") for gloss in glosses]

In [52]:
len(glosses)

4164

In [0]:
with open("speech2signs-2017-nmt/ASLG-PC12/ENG-ASL_Train_0.046.asl") as file_obj:
  separated_lines = file_obj.readlines()
  lines = "".join(separated_lines)
  separated_lines = [line.split() for line in separated_lines]
# WOULD LIKE -> WANT
removed_words = [".", ",", "?", "BE", "TO", "MR"]
replacement_mapping = {
    "X-I": "IX-1p",
    "X-WE": "IX-1p-pl-arc",
    "X-IT": "IX:i",
    "X-HE": "IX:i",
    "X-YOU": "IX-2p",
    "X-Y": "IX-2p", # X-Y is you in the general sense so this might not be good
    "THIS": "IX:i",
    "EU": "ns-EUROPE",
    "EUROPE": "ns-EUROPE",
    "EUROPEAN": "ns-EUROPE",# This might not be right to assume
    "WILL": "FUTURE",
    "NEED": "SHOULD",
    "DESC-NOT": "NOT",
    "DESC-ALSO": "ALSO"
}

# Remopve all DESC- from terms
words = lines.split()
words = [word for word in words if word not in removed_words]
words = [word if word not in replacement_mapping else replacement_mapping[word] for word in words]
words = [word if not word.startswith("DESC-") else word[5:] for word in words]

In [0]:
word_count = {}
for word in words:
  if word in word_count:
    word_count[word] += 1
  else:
    word_count[word] = 1

In [89]:
glosses_set = set(glosses)
sorted_word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)

distinct_words = len(sorted_word_count)
total_words = len(words)
word_overlap = [count for word, count in word_count.items() if word in glosses_set]
distinct_word_overlap = len(word_overlap)
total_word_overlap = sum(word_overlap)

print("Vocabulary coverage in ASLG-PC12: {}".format(distinct_word_overlap / distinct_words))
print("Word coverage in ASLG-PC12: {}".format(total_word_overlap / total_words))

Vocabulary coverage in ASLG-PC12: 0.07809234277028311
Word coverage in ASLG-PC12: 0.5590735139884557


In [90]:
[(word, count) for word, count in sorted_word_count[:100] if word not in glosses_set]

[('AS', 4280),
 ('RE', 3582),
 ('BY', 3552),
 ('WOULD', 3505),
 ('AT', 3384),
 ('SE', 3015),
 ('X-POSS', 2949),
 ('OR', 2833),
 ('COMMISSION', 2700),
 ('HOWEVER', 2210),
 ('UNION', 2175),
 ('REFORE', 2015),
 ('DEBATE', 1804),
 ('X-MY', 1800),
 ('NO', 1769),
 ('X-ITS', 1751),
 ('SO', 1625),
 ('ISSUE', 1589),
 ('COUNCIL', 1514),
 ('THANK', 1510),
 ('ONLY', 1508),
 ('CRISIS', 1380),
 ('POINT', 1233),
 ('AGREEMENT', 1222),
 ('RIGHTS', 1165),
 ('AREA', 1153),
 ('TODAY', 1143),
 ('PROPOSAL', 1091),
 ('BELIEVE', 1082),
 ('SUCH', 1041)]

In [0]:
for gloss in glosses_set:
  if gloss.endswith("+"):
    print(gloss)

In [0]:
sentence_coverage = sorted(
    [sum(word in glosses_set for word in line) / len(line) for line in separated_lines],
    reverse=True
)

In [95]:
pd.Series(sentence_coverage).describe()

count    83619.000000
mean         0.317633
std          0.165227
min          0.000000
25%          0.214286
50%          0.307692
75%          0.400000
max          1.000000
dtype: float64

In [96]:
sum(cov == 1.0 for cov in sentence_coverage)

914